<p>
<img src="https://alanattableau.files.wordpress.com/2014/12/cropped-databender4.png" align=center>
<h1 align=center>The Last Databenders</h1>
<h4 align=center>Sponsored By Gutenberg</h4>
<img src="http://www.gutenberg.org/pics/logo-144x144.png" align=center>
</p>

In [1]:
HEADER_MARKERS = [

      '* START OF THE PROJECT GUTENBERG',
      '* START OF THIS PROJECT GUTENBERG',
      'This etext was prepared by',
      'E-text prepared by',
      'Produced by',
      'Distributed Proofreading Team',
      '*END THE SMALL PRINT',
      '***START OF THE PROJECT GUTENBERG',
      'This etext was produced by',
      '* START OF THE COPYRIGHTED',
      'The Project Gutenberg',
      'http://gutenberg.spiegel.de/ erreichbar.',
      'Project Runeberg publishes',
      'Beginning of this Project Gutenberg',
      'Project Gutenberg Online Distributed',
      'Gutenberg Online Distributed',
      'the Project Gutenberg Online Distributed',
      'Project Gutenberg TEI',
      'This eBook was prepared by',
      'http://gutenberg2000.de erreichbar.',
      'This Etext was prepared by',
      'Gutenberg Distributed Proofreaders',
      'the Project Gutenberg Online Distributed Proofreading Team',
      '**The Project Gutenberg',
      '*SMALL PRINT!',
      'More information about this book is at the top of this file.',
      'tells you about restrictions in how the file may be used.',
      'of the etext through OCR.',
      '**These eBooks Were Prepared By Thousands of Volunteers!**',
      'SERVICE THAT CHARGES FOR DOWNLOAD',
      'We need your donations more than ever!',
      ' * START OF THIS PROJECT GUTENBERG',
      '**     SMALL PRINT!'
    
]
FOOTER_MARKERS = [
    '* END OF THE PROJECT GUTENBERG',
    '* END OF THIS PROJECT GUTENBERG',
    '***END OF THE PROJECT GUTENBERG',
    'End of the Project Gutenberg',
    'End of The Project Gutenberg',
    'by Project Gutenberg',
    'End of Project Gutenberg',
    'End of this Project Gutenberg',
    '        ***END OF THE PROJECT GUTENBERG',
    '* END OF THE COPYRIGHTED',
    'End of this is COPYRIGHTED',
    '*This is a COPYRIGHTED Project Gutenberg Etext, Details Above*',
    'More information about this book is at the top of this file.',
    'We need your donations more than ever!',
    '<<THIS ELECTRONIC VERSION OF',
    'END OF PROJECT GUTENBERG',
    ' End of the Project Gutenberg',
    ' * END OF THIS PROJECT GUTENBERG',
    "THE END"  
]

In [2]:
import urllib2, string, json  # the lib that handles the url stuff
from bs4 import BeautifulSoup
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.sql import Row

def ParseBook(data):
    is_header = False
    is_footer = False
    for line in data:
        for footer in FOOTER_MARKERS:
            if footer in line.strip():
                is_footer = True
                break
        if is_footer:
            break
        if is_header:
            for i in line.split():
                word = i.strip().lower().translate(None, string.punctuation)
                tx.write(word+" ")

        for header in HEADER_MARKERS:
            if header in line.strip():
                is_header = True
                break
    
    """for line in data: # files are iterable
        for i in line.split():
            word = i.strip().lower().translate(None, string.punctuation)
            tx.write(word+" ")"""
                
def FillDict(index, cat):
    tx.write(cat+"\t")
    try:
        data = urllib2.urlopen("http://www.gutenberg.org/files/"+str(index)+"/"+str(index)+"-0.txt") # it's a file like object and works just like a file
        ParseBook(data)
    except urllib2.HTTPError:
        try:
            data = urllib2.urlopen("http://www.gutenberg.org/files/"+str(index)+"/"+str(index)+".txt") # it's a file like object and works just like a file
            ParseBook(data)
        except:
            pass
    tx.write("\n")
    
def FindBookIndex():
    categorise = ['Children']
    for cat in categorise:
        book_index = 0
        print cat
        while book_index < 100:
            try:
                html_doc = urllib2.urlopen("http://www.gutenberg.org/ebooks/search/?start_index="+str(book_index)+"&query="+cat+"+l.english").read()
                soup = BeautifulSoup(html_doc, 'html.parser')
                lis = soup.find_all("li",  { "class" : "booklink" })
                for i in range(len(lis)):
                    index = lis[i].a['href'].split("/")[2]
                    FillDict(index, cat)
                book_index += 26
            except urllib2.HTTPError:
                break
                
tx = open("main_Children.txt", "w")
FindBookIndex()
tx.close()
print "DONEE"

Children
DONEE
